In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
data = pd.read_csv('preprocess.csv',
                  encoding ='ISO-8859-1')
print(data)

      Unnamed: 0           created_at  favorite_count              id_str  \
0              0  2016-12-30 22:18:18           84254  814958820980039680   
1              1  2016-12-30 19:46:55           25336  814920722208296960   
2              2  2016-12-30 19:41:33           97669  814919370711461888   
3              3  2016-12-29 14:54:21           45609  814484710025994240   
4              4  2016-12-28 22:06:28           51857  814231064847728640   
5              5  2016-12-28 14:25:11          117710  814114980983427072   
6              6  2016-12-28 14:19:46          106626  814113616110751744   
7              7  2016-12-28 14:07:13          117900  814110460761018368   
8              8  2016-12-28 03:10:07           78521  813945096269860864   
9              9  2016-12-27 21:52:29           99838  813865160163098624   
10            10  2016-12-27 03:06:59           67470  813581917215977472   
11            11  2016-12-27 02:53:20           71643  813578484572450816   

In [7]:
data_scrub = data.dropna()
print(data_scrub)

      Unnamed: 0           created_at  favorite_count              id_str  \
0              0  2016-12-30 22:18:18           84254  814958820980039680   
1              1  2016-12-30 19:46:55           25336  814920722208296960   
2              2  2016-12-30 19:41:33           97669  814919370711461888   
3              3  2016-12-29 14:54:21           45609  814484710025994240   
4              4  2016-12-28 22:06:28           51857  814231064847728640   
5              5  2016-12-28 14:25:11          117710  814114980983427072   
6              6  2016-12-28 14:19:46          106626  814113616110751744   
7              7  2016-12-28 14:07:13          117900  814110460761018368   
8              8  2016-12-28 03:10:07           78521  813945096269860864   
9              9  2016-12-27 21:52:29           99838  813865160163098624   
10            10  2016-12-27 03:06:59           67470  813581917215977472   
11            11  2016-12-27 02:53:20           71643  813578484572450816   

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_scrub["text_pp"])

true_k = 12
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["Thank you all for coming to this rally in new hampshire, great turnout, big in polls"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["Crooked Hillary is a weak leader! Bad!"])
prediction = model.predict(Y)
print(prediction)
labels = model.labels_

Y = vectorizer.transform(["first im carolina"])
prediction = model.predict(Y)
print(prediction)
#Then get the frequency count of the non-negative labels
counts = np.bincount(labels[labels>=0])

print(counts)

Top terms per cluster:
Cluster 0:
 enjoy
 interviewed
 tonight
 pm
 morning
 week
 wife
 eastern
 tomorrow
 melania
Cluster 1:
 thank
 first
 america
 maga
 im
 support
 great
 carolina
 north
 debate
Cluster 2:
 make
 america
 great
 trump
 2016
 thank
 vote
 safe
 together
 new
Cluster 3:
 let
 america
 make
 never
 work
 great
 hard
 get
 vote
 defeat
Cluster 4:
 new
 hampshire
 york
 poll
 thank
 trump
 fitn
 great
 nh
 2016
Cluster 5:
 trump
 2016
 thank
 vote
 primary
 america
 16
 penny
 join
 tuesday
Cluster 6:
 hillary
 crooked
 clinton
 bad
 bernie
 want
 judgement
 said
 sander
 isi
Cluster 7:
 president
 today
 trump
 obama
 people
 get
 american
 donald
 great
 job
Cluster 8:
 people
 cruz
 ted
 rubio
 lyin
 marco
 great
 like
 trump
 country
Cluster 9:
 trump
 great
 job
 debate
 time
 join
 get
 tonight
 medium
 like
Cluster 10:
 swamp
 drain
 time
 clinton
 truth
 league
 hillary
 debate
 crooked
 big
Cluster 11:
 rt
 league
 debate
 big
 truth
 trump
 http
 vp
 2016
 ð

In [9]:
data_scrub['label_kmeans'] = model.labels_

C:\Users\feder\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
import random

groups=[data_scrub.loc[data_scrub['label_kmeans'] == i] for i in range(true_k)]
for i in range(true_k):
    test=(groups[i].sample(3)["text"])
    print("sampling cluster",i)
    for tweet in test:
        print(tweet)
        print("\n")

sampling cluster 0
Will be interviewed by @StephenAtHome tonight by phone- a late show first @CBS @colbertlateshow. Enjoy! 
#Colbert #LSSC


I will be interviewed tonight on @seanhannity - Enjoy!  10:00 P.M.


I will be interviewed on @oreillyfactor at 8:00 P.M. Enjoy!


sampling cluster 1
Thank you America! #MAGA
https://t.co/FljBVvbwKr https://t.co/ePJW9jLckM


. #RepMikeKelly  Great job on @foxandfriends this morning. Thank you for the nice words!


#AmericaFirst #ImWithYou https://t.co/iEcnnlJ728


sampling cluster 2
"@Ausbiz: @realDonaldTrump Trump is so impressive and no other politician has his passion &amp; energy to #MakeAmericaGreatAgain


Together, we are going to MAKE AMERICA SAFE AND GREAT AGAIN!
https://t.co/pUtN4kFhDK


For the great people of Iowa, find your #IACaucus location at https://t.co/ANvTcZqfOq - so important to vote! #MakeAmericaGreatAgain


sampling cluster 3
LETS MAKE AMERICA GREAT AGAIN!
Schedule &amp; tickets: https://t.co/nU39QHzxxX https://t.co/GUQo1TMsM